In [ ]:
# !pip install librosa

In [1]:
# remove warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import librosa

# Read the Feather file
df = pd.read_feather('CRSP_daily_data_for_project(Technical_Analysis).feather')

In [3]:
from datetime import datetime
df['date']=pd.to_datetime(df['date'], format='%Y-%m-%d')

In [4]:
df.sort_values(by=['date','PERMNO'],inplace=True)

In [5]:
df['DailyRet'] = df.groupby('PERMNO')['Close'].pct_change()
#df['MonthRet'] = df.groupby('PERMNO')['Close'].apply(lambda x: x.shift(-30).pct_change()).reset_index(level=0, drop=True)
df['MonthRet'] = df.groupby('PERMNO')['Close'].apply(lambda x: (x.shift(-30)-x)/x).reset_index(level=0, drop=True)
df

,PERMNO,date,Open,High,Low,Close,Volume,SHROUT,vwretx,ewretx,sprtrn,DailyRet,MonthRet
19071,10006,1969-01-02,NaN,63.25000,63.00000,63.125,3400.0,5641.0,0.001022,0.005752,0.000674,NaN,0.027723
38825,10014,1969-01-02,NaN,16.25000,15.37500,15.375,21100.0,4109.0,0.001022,0.005752,0.000674,NaN,0.097561
164917,10057,1969-01-02,NaN,28.37500,28.12500,28.250,1300.0,2038.0,0.001022,0.005752,0.000674,NaN,0.061947
293514,10102,1969-01-02,NaN,34.12500,33.25000,33.500,12300.0,11036.0,0.001022,0.005752,0.000674,NaN,-0.070896
386105,10137,1969-01-02,NaN,24.62500,24.12500,24.500,2400.0,19292.0,0.001022,0.005752,0.000674,NaN,0.010204
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92907830,93423,2023-12-29,24.92000,25.25000,24.64440,25.080,1306363.0,83537.0,-0.004084,-0.007546,-0.002826,0.004003,NaN
92918058,93426,2023-12-29,33.79000,34.50000,33.79000,34.070,94057.0,12510.0,-0.004084,-0.007546,-0.002826,0.013385,NaN
92926793,93429,2023-12-29,177.85001,179.09000,177.44000,178.560,521913.0,105556.0,-0.004084,-0.007546,-0.002826,0.004049,NaN
92933480,93434,2023-12-29,0.62000,0.71495,0.58500,0.700,169185.0,43039.0,-0.004084,-0.007546,-0.002826,0.166667,NaN


In [6]:
df['DailyRet'].dropna(inplace=True)
df['MonthRet'].dropna(inplace=True)

In [7]:
df = df[~df['DailyRet'].isna()].reset_index(drop=True)
df = df[~df['MonthRet'].isna()].reset_index(drop=True)
df

,PERMNO,date,Open,High,Low,Close,Volume,SHROUT,vwretx,ewretx,sprtrn,DailyRet,MonthRet
0,10006,1969-01-03,NaN,63.75000,63.12500,63.500,3600.0,5641.0,0.000115,0.001501,0.000577,0.005941,0.019685
1,10014,1969-01-03,NaN,15.75000,15.00000,15.125,26500.0,4109.0,0.000115,0.001501,0.000577,-0.016260,0.066116
2,10057,1969-01-03,NaN,28.37500,27.87500,28.250,2600.0,2038.0,0.000115,0.001501,0.000577,0.000000,0.048673
3,10102,1969-01-03,NaN,33.87500,33.37500,33.500,11300.0,11036.0,0.000115,0.001501,0.000577,0.000000,-0.082090
4,10137,1969-01-03,NaN,24.75000,24.37500,24.750,3900.0,19292.0,0.000115,0.001501,0.000577,0.010204,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66590894,93423,2023-11-15,24.74000,24.87000,23.72000,24.030,1441865.0,83520.0,0.001816,0.005472,0.001597,-0.027519,0.043695
66590895,93426,2023-11-15,29.84000,30.19000,29.63000,29.820,52595.0,12581.0,0.001816,0.005472,0.001597,0.006752,0.142522
66590896,93429,2023-11-15,177.39000,178.24001,176.57001,176.980,1031479.0,105517.0,0.001816,0.005472,0.001597,-0.002311,0.008928
66590897,93434,2023-11-15,0.66500,0.71000,0.65000,0.650,29579.0,43039.0,0.001816,0.005472,0.001597,0.000000,0.076923


In [8]:
df=df[df['date'].dt.year>=1998]
df

,PERMNO,date,Open,High,Low,Close,Volume,SHROUT,vwretx,ewretx,sprtrn,DailyRet,MonthRet
22954182,10001,1989-01-03,NaN,6.50000,6.12500,6.31250,0.0,998.0,-0.007737,0.002028,-0.008678,-0.009804,0.029703
22954183,10002,1989-01-03,NaN,8.75000,8.00000,8.37500,200.0,1175.0,-0.007737,0.002028,-0.008678,0.000000,0.029851
22954184,10003,1989-01-03,NaN,19.25000,18.75000,18.75000,4250.0,2194.0,-0.007737,0.002028,-0.008678,-0.013158,-0.066667
22954185,10005,1989-01-03,NaN,0.12500,0.06250,0.09375,0.0,8376.0,-0.007737,0.002028,-0.008678,0.000000,0.000000
22954186,10009,1989-01-03,NaN,11.75000,11.75000,11.75000,200.0,941.0,-0.007737,0.002028,-0.008678,0.044444,-0.063830
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66590894,93423,2023-11-15,24.74000,24.87000,23.72000,24.03000,1441865.0,83520.0,0.001816,0.005472,0.001597,-0.027519,0.043695
66590895,93426,2023-11-15,29.84000,30.19000,29.63000,29.82000,52595.0,12581.0,0.001816,0.005472,0.001597,0.006752,0.142522
66590896,93429,2023-11-15,177.39000,178.24001,176.57001,176.98000,1031479.0,105517.0,0.001816,0.005472,0.001597,-0.002311,0.008928
66590897,93434,2023-11-15,0.66500,0.71000,0.65000,0.65000,29579.0,43039.0,0.001816,0.005472,0.001597,0.000000,0.076923


In [9]:
df['PERMNO+Date'] = list(zip(df['PERMNO'], df['date'].dt.strftime('%Y-%m-%d')))
df

,PERMNO,date,Open,High,Low,Close,Volume,SHROUT,vwretx,ewretx,sprtrn,DailyRet,MonthRet,PERMNO+Date
22954182,10001,1989-01-03,NaN,6.50000,6.12500,6.31250,0.0,998.0,-0.007737,0.002028,-0.008678,-0.009804,0.029703,"(10001, 1989-01-03)"
22954183,10002,1989-01-03,NaN,8.75000,8.00000,8.37500,200.0,1175.0,-0.007737,0.002028,-0.008678,0.000000,0.029851,"(10002, 1989-01-03)"
22954184,10003,1989-01-03,NaN,19.25000,18.75000,18.75000,4250.0,2194.0,-0.007737,0.002028,-0.008678,-0.013158,-0.066667,"(10003, 1989-01-03)"
22954185,10005,1989-01-03,NaN,0.12500,0.06250,0.09375,0.0,8376.0,-0.007737,0.002028,-0.008678,0.000000,0.000000,"(10005, 1989-01-03)"
22954186,10009,1989-01-03,NaN,11.75000,11.75000,11.75000,200.0,941.0,-0.007737,0.002028,-0.008678,0.044444,-0.063830,"(10009, 1989-01-03)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66590894,93423,2023-11-15,24.74000,24.87000,23.72000,24.03000,1441865.0,83520.0,0.001816,0.005472,0.001597,-0.027519,0.043695,"(93423, 2023-11-15)"
66590895,93426,2023-11-15,29.84000,30.19000,29.63000,29.82000,52595.0,12581.0,0.001816,0.005472,0.001597,0.006752,0.142522,"(93426, 2023-11-15)"
66590896,93429,2023-11-15,177.39000,178.24001,176.57001,176.98000,1031479.0,105517.0,0.001816,0.005472,0.001597,-0.002311,0.008928,"(93429, 2023-11-15)"
66590897,93434,2023-11-15,0.66500,0.71000,0.65000,0.65000,29579.0,43039.0,0.001816,0.005472,0.001597,0.000000,0.076923,"(93434, 2023-11-15)"


In [10]:
df['data_cumcount'] = df.groupby('PERMNO').cumcount()


In [11]:
look_back_period = 30
begin_date = '2000-01-01'
end_date = '2002-12-31'
train_test_split_date = '2001-01-01'
# date = df.index.get_level_values('date')
df_temp = df[(df['date']>begin_date)&(df['date']<=end_date)].copy()
# del date
# We only want data point with enough lag information
df_temp = df_temp[df_temp['data_cumcount'] > look_back_period]
# eliminate meanless data points from padding value
df_temp = df_temp[df_temp['MonthRet'] != 0]
df_temp = df_temp.reset_index()

# Generate Pair_PERMNO_date for future indexing
# df_temp['Pair_PERMNO_date'] = list(zip(df_temp['PERMNO'], df_temp['date']))
# take the last day in the month as sample
df_temp['date'] = pd.to_datetime(df_temp['date']).dt.to_period('M').astype(str)
df_temp = df_temp.groupby(['PERMNO','date']).tail(1)

# reset index to make it more convenient for future use
df_temp = df_temp.reset_index()

In [12]:
# df_temp

In [13]:
type(df_temp['date'][0])

str

In [14]:
train_test_split_date

'2001-01-01'

In [15]:
df_train = df_temp[df_temp['date']<train_test_split_date]
df_test = df_temp[df_temp['date']>=train_test_split_date]

In [16]:
label_name = 'MonthRet'
X_train = df_train.drop(label_name, axis=1)
X_test = df_test.drop(label_name, axis=1)

y_train = df_train[label_name]
y_test = df_test[label_name]

In [17]:
# Function: Generate CQT data for each stock
def create_cqt_df(stock_df, n_bins=20):
    stock_returns = stock_df['DailyRet'].values

    # Data preprocessing: Ensure all values are finite
    stock_returns = np.nan_to_num(stock_returns, nan=0.0, posinf=0.0, neginf=0.0)

    sr = 1
    fmin = 0.01  # Minimum frequency
    hop_length = 1
    cqt_result = librosa.cqt(stock_returns, n_bins=n_bins, sr=sr, hop_length=hop_length, fmin=fmin)
    cqt_result_db = librosa.amplitude_to_db(np.abs(cqt_result), ref=np.mean)

    cqt_df = pd.DataFrame(cqt_result_db.T, columns=[f'CQT_{i+1}' for i in range(cqt_result_db.shape[0])])
    cqt_df = cqt_df.loc[:len(stock_df)-1, :]  # Ensure the length matches the original data
    # cqt = pd.concat([stock_df['date'], cqt_df], axis=1)
    # cqt = pd.concat([stock_df.index.get_level_values(1), cqt_df], axis=1)
    cqt_df.set_index(stock_df.index.get_level_values(1).to_flat_index(), inplace=True)
    return cqt_df

# Function: Create rolling windows and retain date indices
def create_rolling_windows_with_dates(data, cqt_window_size, date_to_index):
    windows = []
    date_indices = []
    for i in range(len(data) - cqt_window_size + 1):
        window = data.iloc[i:i + cqt_window_size, 1:].values  # Exclude the date column, keep only CQT data
        windows.append(window)
        start_date = data.iloc[i]['date']
        if start_date in date_to_index:
            date_indices.append(date_to_index[start_date])
    return np.array(windows), date_indices


In [18]:
df.set_index(['PERMNO', 'date'], inplace=True)

In [19]:
df.sort_index(inplace=True)

In [20]:
df

Open       High        Low      Close       Volume  \
PERMNO date                                                                  
10001  1989-01-03        NaN    6.50000    6.12500    6.31250          0.0   
       1989-01-04        NaN    6.50000    6.25000    6.25000       2400.0   
       1989-01-05        NaN    6.50000    6.12500    6.50000       2050.0   
       1989-01-06        NaN    6.12500    6.12500    6.12500        700.0   
       1989-01-09        NaN    6.50000    6.50000    6.50000        200.0   
...                      ...        ...        ...        ...          ...   
93436  2023-11-09  219.75000  220.80000  206.67999  209.98000  141370646.0   
       2023-11-10  210.03000  215.38000  205.69000  214.64999  130818519.0   
       2023-11-13  215.60001  225.39999  211.61011  223.71001  139344959.0   
       2023-11-14  235.03000  238.13499  230.72000  237.41000  149128332.0   
       2023-11-15  239.28999  246.70000  236.45000  242.84000  149733647.0   

                      SHROUT    vwretx    ewretx    sprtrn  DailyRet  \
PERMNO date                                                            
10001  1989-01-03      998.0 -0.007737  0.002028 -0.008678 -0.009804   
       1989-01-04      998.0  0.012782  0.009149  0.014965 -0.009901   
       1989-01-05      998.0  0.002850  0.005571  0.002076  0.040000   
       1989-01-06      998.0  0.003037  0.006378  0.002357 -0.057692   
       1989-01-09      998.0  0.001448  0.002385  0.001105  0.061224   
...                      ...       ...       ...       ...       ...   
93436  2023-11-09  3178921.0 -0.008586 -0.014880 -0.008084 -0.054613   
       2023-11-10  3178921.0  0.013823  0.004011  0.015616  0.022240   
       2023-11-13  3178921.0 -0.000386  0.000257 -0.000836  0.042208   
       2023-11-14  3178921.0  0.022639  0.030881  0.019075  0.061240   
       2023-11-15  3178921.0  0.001816  0.005472  0.001597  0.022872   

                   MonthRet          PERMNO+Date  data_cumcount  
PERMNO date                                                      
10001  1989-01-03  0.029703  (10001, 1989-01-03)              0  
       1989-01-04  0.040000  (10001, 1989-01-04)              1  
       1989-01-05  0.000000  (10001, 1989-01-05)              2  
       1989-01-06  0.040816  (10001, 1989-01-06)              3  
       1989-01-09  0.000000  (10001, 1989-01-09)              4  
...                     ...                  ...            ...  
93436  2023-11-09  0.202686  (93436, 2023-11-09)           3364  
       2023-11-10  0.195481  (93436, 2023-11-10)           3365  
       2023-11-13  0.168656  (93436, 2023-11-13)           3366  
       2023-11-14  0.066425  (93436, 2023-11-14)           3367  
       2023-11-15  0.023225  (93436, 2023-11-15)           3368  

[43636717 rows x 13 columns]

In [21]:
df.isna().sum()

Open             6999989
High                  31
Low                    5
Close                  0
Volume               907
SHROUT                 0
vwretx                 0
ewretx                 0
sprtrn                 0
DailyRet               0
MonthRet               0
PERMNO+Date            0
data_cumcount          0
dtype: int64

In [37]:
import torch
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, whole_data, X, y=None, df_temp=None):
        self.whole_data = whole_data
        self.df_temp = df_temp
        self.X = X #torch.tensor(X, dtype=torch.float32)
        self.y = y if y is not None else None #torch.tensor(y, dtype=torch.float32) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):

        if self.y is not None:
          x = []
          y = []

          permo_date_locator = df.index.get_loc(self.df_temp['PERMNO+Date'][idx])

          #get x given index
          x_temp=create_cqt_df(df.iloc[permo_date_locator-look_back_period+1:permo_date_locator+1]).to_numpy().reshape(-1,1)
          x.append(x_temp)
          #get y given index
          y_temp = df.iloc[permo_date_locator]['MonthRet']
          y.append(y_temp)

          x = torch.tensor(x_temp, dtype=torch.float32)
          y = torch.tensor(y_temp, dtype=torch.float32)
          return x, y

        else:
          x = []
          i = idx
          permo_date_locator = df.index.get_loc(self.df_temp['PERMNO+Date'][idx])
          #get x given index
          x_temp  = create_cqt_df(df.iloc[permo_date_locator-look_back_period+1:permo_date_locator+1]).to_numpy()
          x.append(x_temp)

          x = torch.tensor(x_temp, dtype=torch.float32)

          return x

        '''
    def get_time_series_sample(self, x):
        return x
        '''

In [23]:
# Create dataset
dataset = TimeSeriesDataset(df, X_train, y_train)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
seed = 42

In [25]:
device

device(type='mps')

In [26]:
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

input_size = 1  # Number of features
hidden_size = 50
output_size = 1
num_layers = 2

model = LSTMModel(input_size, hidden_size, output_size, num_layers).to(device)


In [27]:
import torch.optim as optim

# Hyperparameters
num_epochs = 5
learning_rate = 0.001

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(dataloader):
        inputs = inputs.to(device)  # Add feature dimension and move to device
        targets = targets.to(device)  # Add feature dimension and move to device
        # print(inputs.shape, targets.shape)
        
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}')

print('Training finished.')


Epoch [1/5], Step [100/10210], Loss: 0.0323
Epoch [1/5], Step [200/10210], Loss: 0.0402
Epoch [1/5], Step [300/10210], Loss: 0.0777
Epoch [1/5], Step [400/10210], Loss: 0.0670
Epoch [1/5], Step [500/10210], Loss: 0.0617
Epoch [1/5], Step [600/10210], Loss: 0.0557
Epoch [1/5], Step [700/10210], Loss: 0.1158
Epoch [1/5], Step [800/10210], Loss: 0.0518
Epoch [1/5], Step [900/10210], Loss: 0.0647
Epoch [1/5], Step [1000/10210], Loss: 0.1091
Epoch [1/5], Step [1100/10210], Loss: 0.0587
Epoch [1/5], Step [1200/10210], Loss: 0.1489
Epoch [1/5], Step [1300/10210], Loss: 0.1315
Epoch [1/5], Step [1400/10210], Loss: 0.0995
Epoch [1/5], Step [1500/10210], Loss: 0.0292
Epoch [1/5], Step [1600/10210], Loss: 0.1248
Epoch [1/5], Step [1700/10210], Loss: 0.0436
Epoch [1/5], Step [1800/10210], Loss: 0.1111
Epoch [1/5], Step [1900/10210], Loss: 0.0270
Epoch [1/5], Step [2000/10210], Loss: 0.0604
Epoch [1/5], Step [2100/10210], Loss: 0.0384
Epoch [1/5], Step [2200/10210], Loss: 0.1147
Epoch [1/5], Step [

In [28]:
# Create dataset
test_dataset = TimeSeriesDataset(df, X_test, y_test)

# Create DataLoader
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [29]:
torch.save(model, f'models/lstm_model_{datetime.strptime(begin_date, "%Y-%m-%d").strftime("%Y%m%d")}.pth')

In [30]:
model.eval()
running_loss = 0.0
predictions = []
for _, batch in enumerate(test_dataloader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)
    with torch.no_grad():
        y_pred = model(x_batch)
        predictions.append(y_pred.cpu().numpy())
        
        loss = criterion(y_pred, y_batch)
        running_loss += loss.item()
    print(f'Loss: {loss.item()}')
print(f'Average Loss: {running_loss/len(test_dataloader)}')

Loss: 0.1110469400882721
Loss: 0.1740042269229889
Loss: 0.25126856565475464
Loss: 0.08399482816457748
Loss: 0.08664102852344513
Loss: 0.8181102275848389
Loss: 0.0026830926071852446
Loss: 0.001289167208597064
Loss: 0.029870161786675453
Loss: 0.012655836530029774
Loss: 0.011738024652004242
Loss: 0.006779124028980732
Loss: 0.0040651047602295876
Loss: 0.008473629131913185
Loss: 0.02478805184364319
Loss: 0.25900089740753174
Loss: 0.029204748570919037
Loss: 0.062424443662166595
Loss: 0.08595036715269089
Loss: 0.04680795595049858
Loss: 0.036120571196079254
Loss: 0.03408978134393692
Loss: 0.027399176731705666
Loss: 0.04024284705519676
Loss: 0.03819556534290314
Loss: 0.04905988276004791
Loss: 0.039149537682533264
Loss: 0.033184364438056946
Loss: 0.03302169591188431
Loss: 0.041630424559116364
Loss: 0.07799503207206726
Loss: 0.038985781371593475
Loss: 0.031594168394804
Loss: 0.04145638644695282
Loss: 0.039837323129177094
Loss: 0.08896991610527039
Loss: 0.02243732288479805
Loss: 0.0205634310841560

In [31]:
predictions = [i.reshape(-1).tolist() for i in predictions]
# Assuming nested_list is your nested list
predictions = np.array([item for sublist in predictions for item in sublist])
X_test['PredictedMonthRet'] = predictions
X_test

,level_0,index,PERMNO,date,Open,High,Low,Close,Volume,SHROUT,vwretx,ewretx,sprtrn,DailyRet,PERMNO+Date,data_cumcount,PredictedMonthRet
163349,3323226,42551764,83193,2001-02,0.56,0.81,0.56000,0.77,181500.0,25018.0,0.003739,0.001923,0.005461,0.350877,"(83193, 2001-02-01)",1234,0.022918
163350,3324673,42553236,86937,2001-02,8.27,8.29,8.27000,8.29,83800.0,117334.0,0.003739,0.001923,0.005461,0.002418,"(86937, 2001-02-01)",415,0.022921
163351,3327711,42556324,76363,2001-02,1.15,1.15,1.14000,1.14,46000.0,37061.0,-0.018535,-0.009245,-0.017474,-0.037975,"(76363, 2001-02-02)",2738,0.022926
163352,3328064,42556681,77663,2001-02,0.50,0.50,0.46875,0.50,59504.0,16711.0,-0.018535,-0.009245,-0.017474,0.000000,"(77663, 2001-02-02)",2187,0.022926
163353,3331048,42559723,87436,2001-02,5.83,6.06,5.83000,5.90,340600.0,43234.0,-0.018535,-0.009245,-0.017474,-0.026403,"(87436, 2001-02-02)",303,0.022926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292123,5958547,45220127,92655,2002-12,83.61,83.94,82.61000,83.50,1149200.0,304453.0,0.001689,0.010081,0.000489,-0.001196,"(92655, 2002-12-31)",3531,0.022923
292124,5958548,45220128,92663,2002-12,16.39,17.10,15.75000,16.18,124658.0,25094.0,0.001689,0.010081,0.000489,-0.005532,"(92663, 2002-12-31)",3531,0.022925
292125,5958549,45220129,92807,2002-12,2.70,2.89,2.61000,2.85,31527.0,13898.0,0.001689,0.010081,0.000489,0.021139,"(92807, 2002-12-31)",3531,0.022898
292126,5958550,45220130,92874,2002-12,NaN,7.15,6.47000,6.81,0.0,4762.0,0.001689,0.010081,0.000489,-0.021411,"(92874, 2002-12-31)",3531,0.022908


In [32]:
X_test.set_index(['PERMNO', 'date'], inplace=True)

In [33]:
X_test.sort_index(inplace=True)

In [ ]:
# X_test

level_0     index       Open       High        Low      Close  \
PERMNO date                                                                     
10001  2016-02  1054210  61332124    7.85000    7.91000    7.75000    7.86000   
       2016-03  1132582  61414384    7.88000    7.99000    7.81000    7.81000   
       2016-04  1207027  61492722    7.33000    7.45000    7.25000    7.30000   
       2016-05  1281685  61570884    7.08000    7.16000    7.08000    7.14000   
       2016-06  1359226  61652583    6.99000    7.08000    6.97010    6.99000   
...                 ...       ...        ...        ...        ...        ...   
93436  2016-08  1513824  61815429  210.42999  212.60001  208.64999  212.00999   
       2016-09  1586259  61892696  202.21001  204.98000  199.55000  204.03000   
       2016-10  1658626  61970006  202.49001  202.49001  195.81000  197.73000   
       2016-11  1731201  62047256  191.00000  191.89000  187.50000  189.39999   
       2016-12  1803307  62124080  216.30000  217.50000  211.67999  213.69000   

                   Volume    SHROUT    vwretx    ewretx    sprtrn  DailyRet  \
PERMNO date                                                                   
10001  2016-02     7548.0   10505.0 -0.005969  0.001095 -0.008121  0.002551   
       2016-03    10293.0   10508.0 -0.000879  0.001552 -0.002040 -0.010139   
       2016-04    17891.0   10512.0 -0.004496 -0.003827 -0.005063 -0.004093   
       2016-05    17665.0   10512.0 -0.000393  0.003267 -0.001000  0.011331   
       2016-06    95131.0   10512.0  0.013151  0.009795  0.013565  0.004310   
...                   ...       ...       ...       ...       ...       ...   
93436  2016-08  3276548.0  149239.0 -0.002719 -0.003870 -0.002376  0.003170   
       2016-09  2586273.0  149825.0  0.007800  0.007513  0.007968  0.016592   
       2016-10  4709373.0  149891.0  0.000418 -0.001165 -0.000122 -0.011202   
       2016-11  3547330.0  149964.0 -0.001744 -0.001042 -0.002653 -0.000897   
       2016-12  4642620.0  161561.0 -0.004070 -0.001834 -0.004637 -0.004611   

                        PERMNO+Date  PredictedMonthRet  
PERMNO date                                             
10001  2016-02  (10001, 2016-02-29)           0.002194  
       2016-03  (10001, 2016-03-31)           0.002640  
       2016-04  (10001, 2016-04-29)           0.002670  
       2016-05  (10001, 2016-05-31)           0.002579  
       2016-06  (10001, 2016-06-30)           0.002705  
...                             ...                ...  
93436  2016-08  (93436, 2016-08-31)           0.002206  
       2016-09  (93436, 2016-09-30)           0.002855  
       2016-10  (93436, 2016-10-31)           0.002943  
       2016-11  (93436, 2016-11-30)           0.001879  
       2016-12  (93436, 2016-12-30)           0.002616  

[40126 rows x 14 columns]

In [34]:
X_test.reset_index(inplace=True)

In [37]:
# predictions

[array([0.00272575], dtype=float32),
 array([0.00273754], dtype=float32),
 array([0.00255124], dtype=float32),
 array([0.00237295], dtype=float32),
 array([0.00223523], dtype=float32),
 array([0.00266004], dtype=float32),
 array([0.00171578], dtype=float32),
 array([0.00277792], dtype=float32),
 array([0.00341243], dtype=float32),
 array([0.00240898], dtype=float32),
 array([0.00297973], dtype=float32),
 array([0.0031635], dtype=float32),
 array([0.00281611], dtype=float32),
 array([0.00279696], dtype=float32),
 array([0.00183662], dtype=float32),
 array([0.00285402], dtype=float32),
 array([0.00336087], dtype=float32),
 array([0.00329336], dtype=float32),
 array([0.00267316], dtype=float32),
 array([0.00185765], dtype=float32),
 array([0.00273539], dtype=float32),
 array([0.00245713], dtype=float32),
 array([0.00279895], dtype=float32),
 array([0.00260926], dtype=float32),
 array([0.00289328], dtype=float32),
 array([0.00285513], dtype=float32),
 array([0.00346119], dtype=float32),
 a

In [35]:
X_test[['PERMNO', 'date', 'PredictedMonthRet', 'DailyRet']].to_csv(f'predictions/lstm_predictions_{datetime.strptime(begin_date, "%Y-%m-%d").strftime("%Y%m%d")}.csv')

In [36]:
X_test.date.max()

'2002-12'

In [46]:
from dateutil.relativedelta import relativedelta


def train_model_annually(start_date, end_date):
    # models = []
    # iterate between start_date and end_date with 1 year step size
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    while start_date < end_date:
        next_year = start_date + relativedelta(years=2)
        
        if next_year > end_date:
            next_year = end_date
            break
        
        train_test_split_date = start_date + relativedelta(years=1)

        df.reset_index(inplace=True)

        df_temp = df[(df['date']>=start_date)&(df['date']<=next_year)].copy()
        df_temp = df_temp[df_temp['MonthRet'] != 0]
        df_temp = df_temp.reset_index()
        df_temp['date'] = pd.to_datetime(df_temp['date']).dt.to_period('M').astype(str)
        df_temp = df_temp.groupby(['PERMNO','date']).tail(1)

        df.set_index(['PERMNO', 'date'], inplace=True)
        df.sort_index(inplace=True)
        
        df_temp = df_temp.reset_index()

        df_train = df_temp[df_temp['date']<train_test_split_date.strftime('%Y-%m-%d')]
        df_test = df_temp[df_temp['date']>=train_test_split_date.strftime('%Y-%m-%d')]
        
        label_name = 'MonthRet'
        X_train = df_train.drop(label_name, axis=1)
        X_test = df_test.drop(label_name, axis=1)

        y_train = df_train[label_name]
        y_test = df_test[label_name]

        # Create dataset
        dataset = TimeSeriesDataset(df, X_train, y_train, df_temp)

        # Create DataLoader
        dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

        
        model = LSTMModel(input_size, hidden_size, output_size, num_layers).to(device)

        num_epochs = 2
        learning_rate = 0.001

        # Loss and optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)


        # Training loop
        for epoch in range(num_epochs):
            for i, (inputs, targets) in enumerate(dataloader):
                inputs = inputs.to(device)  
                targets = targets.to(device)
                
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                if (i+1) % 10 == 0:
                    print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}')

        print('Training finished.')

        test_dataset = TimeSeriesDataset(df, X_test, y_test, df_temp)
        test_dataloader = DataLoader(test_dataset, batch_size=16)

        torch.save(model, f'models/lstm_model_{start_date.strftime("%Y%m%d")}.pth')

        model.eval()
        running_loss = 0.0
        predictions = []
        for _, batch in enumerate(test_dataloader):
            x_batch, y_batch = batch[0].to(device), batch[1].to(device)
            with torch.no_grad():
                y_pred = model(x_batch)
                predictions.append(y_pred.cpu().numpy())
                
                loss = criterion(y_pred, y_batch)
                running_loss += loss.item()
            print(f'Loss: {loss.item()}')
        print(f'Average Loss: {running_loss/len(test_dataloader)}')

        predictions = [i.reshape(-1).tolist() for i in predictions]
        # Assuming nested_list is your nested list
        predictions = np.array([item for sublist in predictions for item in sublist])

        X_test['PredictedMonthRet'] = predictions

        X_test.set_index(['PERMNO', 'date'], inplace=True)
        X_test.sort_index(inplace=True)
        X_test.reset_index(inplace=True)
        
        X_test[['PERMNO', 'date', 'PredictedMonthRet', 'DailyRet']].to_csv(f'predictions/lstm_predictions_{start_date.strftime("%Y%m%d")}.csv')


        start_date = train_test_split_date



In [47]:
begin_date = '2001-01-01'
end_date = '2010-12-31'
train_model_annually(begin_date, end_date)

Epoch [2/2], Step [2610/4781], Loss: 0.0382
Epoch [2/2], Step [2620/4781], Loss: 0.0529
Epoch [2/2], Step [2630/4781], Loss: 0.0425
Epoch [2/2], Step [2640/4781], Loss: 0.0901
Epoch [2/2], Step [2650/4781], Loss: 0.0445
Epoch [2/2], Step [2660/4781], Loss: 0.0163
Epoch [2/2], Step [2670/4781], Loss: 0.0444
Epoch [2/2], Step [2680/4781], Loss: 0.0505
Epoch [2/2], Step [2690/4781], Loss: 0.0732
Epoch [2/2], Step [2700/4781], Loss: 0.0366
Epoch [2/2], Step [2710/4781], Loss: 0.0252
Epoch [2/2], Step [2720/4781], Loss: 0.0287
Epoch [2/2], Step [2730/4781], Loss: 0.0373
Epoch [2/2], Step [2740/4781], Loss: 0.0248
Epoch [2/2], Step [2750/4781], Loss: 0.0926
Epoch [2/2], Step [2760/4781], Loss: 0.1210
Epoch [2/2], Step [2770/4781], Loss: 0.0689
Epoch [2/2], Step [2780/4781], Loss: 0.0818
Epoch [2/2], Step [2790/4781], Loss: 0.0498
Epoch [2/2], Step [2800/4781], Loss: 0.0180
Epoch [2/2], Step [2810/4781], Loss: 0.0221
Epoch [2/2], Step [2820/4781], Loss: 0.0744
Epoch [2/2], Step [2830/4781], L

KeyboardInterrupt: 

In [42]:
df.head()

Open   High    Low   Close  Volume  SHROUT    vwretx  \
PERMNO date                                                               
10001  1989-01-03   NaN  6.500  6.125  6.3125     0.0   998.0 -0.007737   
       1989-01-04   NaN  6.500  6.250  6.2500  2400.0   998.0  0.012782   
       1989-01-05   NaN  6.500  6.125  6.5000  2050.0   998.0  0.002850   
       1989-01-06   NaN  6.125  6.125  6.1250   700.0   998.0  0.003037   
       1989-01-09   NaN  6.500  6.500  6.5000   200.0   998.0  0.001448   

                     ewretx    sprtrn  DailyRet  MonthRet  \
PERMNO date                                                 
10001  1989-01-03  0.002028 -0.008678 -0.009804  0.029703   
       1989-01-04  0.009149  0.014965 -0.009901  0.040000   
       1989-01-05  0.005571  0.002076  0.040000  0.000000   
       1989-01-06  0.006378  0.002357 -0.057692  0.040816   
       1989-01-09  0.002385  0.001105  0.061224  0.000000   

                           PERMNO+Date  data_cumcount  
PERMNO date                                            
10001  1989-01-03  (10001, 1989-01-03)              0  
       1989-01-04  (10001, 1989-01-04)              1  
       1989-01-05  (10001, 1989-01-05)              2  
       1989-01-06  (10001, 1989-01-06)              3  
       1989-01-09  (10001, 1989-01-09)              4